In [1]:
import _harmonization_utils as har_utils
import _lrtp_utils as lrtp_utils
import _sb1_utils as sb1_utils
import _specific_list_utils
import _state_rail_plan_utils as srp_utils
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
import random
import string

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_list/"

### LRTP/LOST

In [75]:
def generate_alphanumeric_ids(df:pd.DataFrame, id_length=8):
    """
    Generate alphanumeric IDs for each row in a Pandas DataFrame and add them to a new column.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - id_column_name (str): Name of the new column to store the generated IDs (default is 'alphanumeric_id').
    - id_length (int): Length of the generated ID (default is 8).

    Returns:
    - pd.DataFrame: DataFrame with the new column containing alphanumeric IDs.
    """
    characters = string.ascii_letters + string.digits
    df['project_number'] = [''.join(random.choice(characters) for _ in range(id_length)) for _ in range(len(df))]
    return df


In [76]:
lrtp_lost_df, lrtp_lost_gdf = lrtp_utils.all_mpo(True)

/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

96 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:720: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sandag.cost2020m.str.replace("$", "")
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


65 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


360 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


68 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

In [77]:
lrtp_lost_df = generate_alphanumeric_ids(lrtp_lost_df, 8)

In [78]:
lrtp_lost_df.head(2)

,project_title,lead_agency,project_year,project_description,total_project_cost,geometry,city,county,data_source,notes,project_number
0,Carmel To Pebble Beach Bike/Ped Facility,Ambag,None,Construct Class I Or Class Ii Bike Facility.,86000.00,None,None,Santa Cruz,Ambag Lrtp,,91UKEzRW
1,Sr 1 Carmel Corridor Between Carmel River Bridge And Carpenter Street,Ambag,None,Provide Accomodation For Bicyclists Along State Route 1 Bike Route.,500000.00,None,None,Santa Cruz,Ambag Lrtp,,K2QvYDoG


In [224]:
def separate_out_df(df:pd.DataFrame, columns_to_keep: list)-> pd.DataFrame:
    """
    Subset the column, drop any rows 
    in which the values are Nan or "None."
    """
    # Subset
    df2 = df[columns_to_keep]
    
    # Fill in missing values
    df2 = df2.fillna('none')
    
    # Remove project_number and keep only cols of interest
    columns_to_keep.remove('project_number')
    
    # Drop rows that are nan or "None" based on how many columns are listed
    if len(columns_to_keep) == 1:
        df2 = df2.dropna(how = "any")
        df2 = df2[df2.applymap(lambda x: x.lower() if isinstance(x, str) else x) != 'none'].dropna()
        
    # If there are more than one column to separate out,
    # keep any row that has a non-null value 
    else:
        df2 = df2.dropna(how = "all", subset = columns_to_keep)
    return df2

In [117]:
lrtp_lost_county = separate_out_df(lrtp_lost_df, ['project_number', 'county'])

In [118]:
lrtp_lost_df.shape

(16276, 11)

In [119]:
lrtp_lost_county.shape

(4012, 2)

In [120]:
lrtp_lost_city = separate_out_df(lrtp_lost_df, ['project_number', 'city'])

In [121]:
lrtp_lost_city.shape

(745, 2)

In [122]:
lrtp_lost_agency = separate_out_df(lrtp_lost_df, ['project_number', 'lead_agency'])

In [123]:
lrtp_lost_agency.shape

(16276, 2)

In [124]:
lrtp_lost_agency.sample()

,project_number,lead_agency
13,8xH5shJ9,Madera Ctc


In [125]:
lrtp_lost_geo = separate_out_df(lrtp_lost_df, ['project_number', 'geometry'])

In [126]:
lrtp_lost_geo.shape, lrtp_lost_gdf.shape

((1357, 2), (1355, 10))

In [127]:
lrtp_to_drop = ['county', 'city', 'lead_agency', 'geometry']

In [128]:
# lrtp_lost_df = lrtp_lost_df.drop(columns = lrtp_to_drop)

In [129]:
lrtp_lost_df.sample()

,project_title,lead_agency,project_year,project_description,total_project_cost,geometry,city,county,data_source,notes,project_number
129,Bus Expansion #6,Madera Ctc,NaN,Purchase 1 Dar Vehicle,339355.00,None,None,None,Madera Ctc Lrtp,"Category: Public Transit, Location: Nan",s0mlY5ts


### SB1

In [130]:
sb1_df = sb1_utils.load_sb1()

/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gdf[i]
/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  gdf[i]


0 rows are mising geometry
7917 rows contain invalid geography


In [131]:
sb1_df = generate_alphanumeric_ids(sb1_df, 8)

#### Check that assemblydistrict and assemblycode are the same values

In [151]:
sb1_df['assembly_same'] = sb1_df.assemblycode == sb1_df.assemblydistrict

In [153]:
sb1_df.assembly_same.value_counts()

True     7217
False    1969
Name: assembly_same, dtype: int64

In [152]:
sb1_df['senate_same'] = sb1_df.senatedistrict == sb1_df.senatecode

In [154]:
sb1_df.senate_same.value_counts()

True     6950
False    2236
Name: senate_same, dtype: int64

In [132]:
sb1_county = separate_out_df(sb1_df, ['project_number', 'countyname'])

#### Need to figure out a way to get values delinated by commas into its own row

In [215]:
def explode_dataframe(df:pd.DataFrame, column_to_explode:str)-> pd.DataFrame:
    df['Column2'] = df[column_to_explode].apply(lambda x: [int(i) if i.isdigit() else i for i in x.replace(',', '').split()])
    df = df.drop(columns = [column_to_explode])
    df2 = df.explode('Column2')
    df2 = df2.rename(columns = {'Column2': column_to_explode})
    return df2

In [133]:
sb1_assembly = separate_out_df(sb1_df, ['project_number', 'assemblydistrict'])

In [219]:
sb1_assembly = explode_dataframe(sb1_assembly, 'assemblydistrict')

In [216]:
sb1_senate = separate_out_df(sb1_df, ['project_number', 'senatedistrict'])

In [217]:
sb1_senate = explode_dataframe(sb1_senate, 'senatedistrict')

In [135]:
sb1_city = separate_out_df(sb1_df, ['project_number', 'cityname'])

In [136]:
sb1_geo = separate_out_df(sb1_df, ['project_number', 'geometry'])

In [137]:
sb1_awards = separate_out_df(sb1_df, ['project_number', 'projprogram'])

In [147]:
sb1_awards.shape, sb1_geo.shape, sb1_city.shape

((9186, 2), (1585, 2), (6696, 2))

#### Fix this so that a row is kept even if all three of the non project-number rows are nan

In [138]:
sb1_agencies = separate_out_df(sb1_df, ['project_number', 'projagency', 'appagencyname', 'impagencyname'])

In [139]:
sb1_agencies.shape

(8130, 4)

### LP2000

In [221]:
def load_lp2000(file: str):
    
    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    ).drop(columns=["project_label_name"])

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    ).drop(columns=["project_label_name"])

    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )
    
    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )
    
    return df_project, df_county, df_district, df_award, df_phase

In [222]:
lp2000_project, lp2000_county, lp2000_district, lp2000_award, lp2000_phase = load_lp2000("LP2000_projects.xlsx")

### CTIPS

In [225]:
def load_ctips(file: str):
  
    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    )

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    )

    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )
    
    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )
    
    df_house = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="us_house")
    )
    
    df_senate = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="senate")
    )
    
    df_assembly = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="assembly")
    )
    return df_project, df_county, df_district, df_phase, df_award, df_house, df_senate, df_assembly

In [226]:
ctips_project, ctips_county, ctips_district, ctips_phase, ctips_award, ctips_house, ctips_senate, ctips_assembly = load_ctips('CTIPS.xlsx')

In [232]:
ctips_project.sample()

,ctips_id,const_date,countyid,countyid2,countyid3,document,docyear,ea_number,high_ver,high_offcl,needpurpose,progcode1,ppno,proj_desc,postmiles1,pm1b,pm2b,pm3b,pm1a,pm2a,pm3a,route1,route2,route3,rtl,title,sponsoring_agency,agency_name,implpse_agency,implrw_agency,implcon_agency,implpse_agency_1
26476,21400000568,datetime64[ns],STA,None,None,FTIP,2010,None,1,1,None,None,None,Grouped Projects For Purchase Of Support Vehicles - Projects Are Consistent With 40 Cfr Part 93.126 Exempt Tables 2 And Tables 3 Categories,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,None,datetime64[ns],Grouped Projects For Purchase Of Support Vehicles,Various Agencies,NaN,NaN,NaN,NaN,NaN


### State Rail Plan

In [227]:
srp_df = har_utils.load_state_rail_plan()

In [229]:
srp_df = generate_alphanumeric_ids(srp_df, 8)

In [230]:
srp_df.sample()

,project_time_horizon,project_name,project_description,lead_agency,total_project_cost,srp_region,corridor,sub_corridor_node_1,sub_corridor_node_2,itsp_corridor,project_category,project_number
32,Near Term,San Luis Obispo (Central Coast) Layover Facility (Full Build Out),"Project will construct a new and expanded layover facility in San Luis Obispo that will improve intercity passenger rail service. The Pacific Surfliner would be able to improve the ridership, revenue, and expand service through additional layover capacity. The project will facilitate the maintenance of equipment mid-route and at route terminus.",**Multiple,77458000,Southern California Megaregion,Central Coast to Los Angeles Corridor,San Luis Obispo,Goleta,South Coast - Central Coast,Capital,TWrUKFmm


In [231]:
srp_df_agency = separate_out_df(srp_df, ['project_number', 'lead_agency'])